In [ ]:
import requests, pymongo

In [ ]:
connectionStr = "mongodb+srv://<username>:<password>@testcorpus.tj5ah.mongodb.net/testDatabase?retryWrites=true&w=majority"

In [ ]:
myClient = pymongo.MongoClient(connectionStr, serverSelectionTimeoutMS = 15000)

In [ ]:
try:
    print(myClient.server_info())
except Exception:
    print("Unable to establish connection to the server!")

In [ ]:
mydb = myClient.testDatabase

In [ ]:
collectionName = input("Type in the name of the collection: ")

In [ ]:
myCollection = mydb[collectionName]

In [ ]:
projectName = input("Type in the name of the project: ")

In [ ]:
annotator = input("Type in the annotator's user name: ")

In [ ]:
tagtogAPIUrl = "https://www.tagtog.net/-api/documents/v1"

In [ ]:
auth = requests.auth.HTTPBasicAuth(username = "my_username", password = "my_password!")

In [ ]:
file = open("recordIDsToConsider.txt", "r")

In [ ]:
listOfRecordsToConsider = []
assignedRecords = []

In [ ]:
for line in file:
    if len(line) != 0:
        listOfRecordsToConsider.append(line.rstrip())

In [ ]:
file.close()

In [ ]:
len(listOfRecordsToConsider)

In [ ]:
numOfRecordsToAssign = int(input("Type in the number of records to assign to the annotator: "))

In [ ]:
counter = 0
for document in myCollection.find():
    if (document["_id"] in listOfRecordsToConsider) and (counter < numOfRecordsToAssign):
        counter += 1
        print(document["_id"])
        params = {"project": projectName, 
            "owner": "herculespan", 
            "output": "ann.json",
            "folder": "docsFolder",
            "filename": document["_id"],
            "format": "verbatim",
            "distributeToMembers":annotator
            }
        payload = {"text": document["citation_abstract"]}
        response = requests.post(tagtogAPIUrl, params = params, auth = auth, data = payload)
        listOfRecordsToConsider.remove(document["_id"])
        assignedRecords.append(document["_id"])

In [ ]:
print(counter)

In [ ]:
file = open("recordIDsToConsider.txt", "w")

In [ ]:
for recordID in listOfRecordsToConsider:
    file.write(recordID + "\n")
file.close()

In [ ]:
fileName = "records_assigned_to_" + annotator + ".txt"
file = open(fileName, "w")
for recordID in assignedRecords:
    file.write(recordID + "\n")
file.close()